# Step 1: Import necessary libraries


In [33]:
import os
import googlemaps
import sys

# Add the project root to sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), "../../.."))
if project_root not in sys.path:
    sys.path.append(project_root)
from app.database import create_connection

# Step 2: Set up Google Maps API key

In [47]:
from app.config import Config
GOOGLE_MAPS_API_KEY = Config.GOOGLE_MAPS_API_KEY  # Load from environment variable
api_key = googlemaps.Client(key=GOOGLE_MAPS_API_KEY)

# Step 3: Fetch coordinates using Google Maps API

In [48]:
import requests
def format_address(name):
    """
    Formats the address by combining name and location for the API request.
    """
    formatted = f"{name}, York University, Toronto, ON, Canada".replace("&", "and").replace(",", "").replace(" ", "+")
    return formatted

def get_coordinates(address):
    """
    Fetches the latitude and longitude for a given address using Google Maps Geocoding API.
    """
    if not api_key:
        raise Exception("GOOGLE_MAPS_API_KEY not set in environment variables.")
    
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {"address": address, "key": GOOGLE_MAPS_API_KEY}
    response = requests.get(base_url, params=params)

    if response.status_code != 200:
        print(f"HTTP Error {response.status_code} for address {address}")
        return None, None

    data = response.json()
    if data["status"] != "OK":
        print(f"Geocoding API error for {address}: {data['status']}")
        return None, None

    location = data["results"][0]["geometry"]["location"]
    return location["lat"], location["lng"]

# Step 4: Update location data in the database


In [57]:
import time
def update_location_for_table(table_name, id_column, name_column, location_column):
    """
    Iterates through all rows in the table, retrieves coordinates, and updates the location.
    """
    print(f"Updating locations for table: {table_name}...")

    connection = create_connection()
    if connection is None:
        print("Failed to connect to the database.")
        return

    try:
        cursor = connection.cursor(dictionary=True)
        # Fetch records
        query = f"SELECT {id_column}, {name_column}, {location_column} FROM {table_name}"
        print(f"Executing query: {query}")
        cursor.execute(query)
        rows = cursor.fetchall()
        print(f"Fetched rows: {rows}")

        for row in rows:
            record_id = row[id_column]
            name = row[name_column]
            location = row[location_column]

            print(f"Processing Record ID: {record_id}, Name: {name}, Existing Location: {location}")
            
            # Format address for API call
            formatted_address = format_address(name)
            print(f"Formatted address: {formatted_address}")

            lat, lng = get_coordinates(formatted_address)
            if lat is None or lng is None:
                print(f"Coordinates not found for {name}. Skipping...")
                continue
            
            print(f"Fetched Coordinates for {name}: ({lat}, {lng})")
            
            # Update the database
            update_query = f"UPDATE {table_name} SET {location_column} = ST_GeomFromText('POINT(%s %s)') WHERE {id_column} = %s"
            cursor.execute(update_query, (lng, lat, record_id))  # Note: lng first, then lat
            connection.commit()
            print(f"Updated {table_name} ID {record_id} with coordinates: {lat}, {lng}")
            time.sleep(0.2)  # To avoid hitting rate limits
            
    except Exception as e:
        print(f"Error updating {table_name}: {e}")
    finally:
        print(f"Finished updating {table_name}.")

def fetch_and_print_locations_for_table(table_name, id_column, name_column, location_column):
    """
    Iterates through all rows in the table, retrieves coordinates, and prints them without updating the database.
    """
    print(f"Fetching and printing locations for table: {table_name}...")

    connection = create_connection()
    if connection is None:
        print("Failed to connect to the database.")
        return

    try:
        cursor = connection.cursor(dictionary=True)
        # Fetch records
        query = f"SELECT {id_column}, {name_column}, {location_column} FROM {table_name}"
        cursor.execute(query)
        rows = cursor.fetchall()

        for row in rows:
            record_id = row[id_column]
            name = row[name_column]
            location = row[location_column]

            formatted_address = format_address(name)
            print(f"Fetching coordinates for: {formatted_address}")

            lat, lng = get_coordinates(formatted_address)
            if lat is None or lng is None:
                print(f"Coordinates not found for {name}. Skipping...")
                continue

            # Print the results
            print(f"Record ID: {record_id}, Name: {name}, Coordinates: ({lat}, {lng})")
            time.sleep(0.2)  # To avoid hitting rate limits
    except Exception as e:
        print(f"Error fetching locations for {table_name}: {e}")
    finally:
        print(f"Finished fetching locations for {table_name}.")

# Step 5: Main script


In [ ]:

# Fetch and print libraries
fetch_and_print_locations_for_table("libraries", "library_id", "name", "location")
# Fetch and print cafes
fetch_and_print_locations_for_table("cafes", "cafe_id", "name", "location")
# Fetch and print buildings
fetch_and_print_locations_for_table("buildings", "building_id", "building_name", "location")

Fetching and printing locations for table: libraries...
Fetching coordinates for: Scott+Library+York+University+Toronto+ON+Canada
Record ID: 1, Name: Scott Library, Coordinates: (43.7723418, -79.5055508)
Fetching coordinates for: Bronfman+Business+Library+York+University+Toronto+ON+Canada
Record ID: 2, Name: Bronfman Business Library, Coordinates: (43.7734232, -79.4987993)
Fetching coordinates for: Steacie+Science+and+Engineering+Library+York+University+Toronto+ON+Canada
Record ID: 3, Name: Steacie Science and Engineering Library, Coordinates: (43.773608, -79.505845)
Finished fetching locations for libraries.
Fetching and printing locations for table: cafes...
Fetching coordinates for: La+Prep+-+York+Lanes+York+University+Toronto+ON+Canada
Record ID: 1, Name: La Prep - York Lanes, Coordinates: (43.7742143, -79.5018478)
Fetching coordinates for: Break+Cafe+-+Second+Student+Center+York+University+Toronto+ON+Canada
Record ID: 2, Name: Break Cafe - Second Student Center, Coordinates: (43.7

In [58]:
# Update libraries
update_location_for_table("libraries", "library_id", "name", "location")
# Update cafes
update_location_for_table("cafes", "cafe_id", "name", "location")
# Update buildings
update_location_for_table("buildings", "building_id", "building_name", "location")



Updating locations for table: libraries...
Executing query: SELECT library_id, name, location FROM libraries
Fetched rows: [{'library_id': 1, 'name': 'Scott Library', 'location': ''}, {'library_id': 2, 'name': 'Bronfman Business Library', 'location': ''}, {'library_id': 3, 'name': 'Steacie Science and Engineering Library', 'location': ''}]
Processing Record ID: 1, Name: Scott Library, Existing Location: 
Formatted address: Scott+Library+York+University+Toronto+ON+Canada
Fetched Coordinates for Scott Library: (43.7723418, -79.5055508)
Updated libraries ID 1 with coordinates: 43.7723418, -79.5055508
Processing Record ID: 2, Name: Bronfman Business Library, Existing Location: 
Formatted address: Bronfman+Business+Library+York+University+Toronto+ON+Canada
Fetched Coordinates for Bronfman Business Library: (43.7734232, -79.4987993)
Updated libraries ID 2 with coordinates: 43.7734232, -79.4987993
Processing Record ID: 3, Name: Steacie Science and Engineering Library, Existing Location: 
Form